In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import (Matrix,
                   sin,
                   cos,
                   symbols,
                   diff,
                   simplify,
                   Eq,
                   linear_eq_to_matrix)
from sympy.physics.mechanics import dynamicsymbols

In [2]:
# rho = np.array([0.5, 0.5, 0.5])
# m = np.array([250., 25., 25.])
# I = np.array([25., 2.5, 2.5])
# bs = b1 = b2 = 0. # parameters for non-conservative forces

C = lambda th: Matrix([[cos(th), -sin(th)], [sin(th), cos(th)]])
D = lambda th: Matrix([[sin(th), cos(th)], [-cos(th), sin(th)]])

In [3]:
t = symbols('t')

th_s, th_1, th_2 = dynamicsymbols('theta_s theta_1 theta_2')
dth_s, dth_1, dth_2 = dynamicsymbols('theta_s theta_1 theta_2', 1)
ddth_s, ddth_1, ddth_2 = dynamicsymbols('theta_s theta_1 theta_2', 2)

q = Matrix([th_s, th_1, th_2])
qdot = Matrix([dth_s, dth_1, dth_2])
qddot = Matrix([ddth_s, ddth_1, ddth_2])

y = q.row_insert(3, qdot)
ydot = qdot.row_insert(3, qddot)

tau_s = symbols('tau_s')
tau_1 = symbols('tau_1')
tau_2 = symbols('tau_2')

tau = Matrix([tau_s, tau_1, tau_2])

rho = symbols('rho_s rho_1 rho_2')
m = symbols('m_s m_1 m_2')
I = symbols('I_s I_1 I_2')
b = symbols('b_s b_1 b_2')


r_s = Matrix([0., 0.])
r_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([rho[1], 0.]))
r_2 = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([rho[2], 0.]))

v_s = diff(r_s, t)
v_1 = diff(r_1, t)
v_2 = diff(r_2, t)

T = 0.5*m[0]*v_s.dot(v_s) + 0.5*m[1]*v_1.dot(v_1) + 0.5*m[2]*v_2.dot(v_2) + \
    0.5*I[0]*y[3]**2 + 0.5*I[1]*y[4]**2 + 0.5*I[2]*y[5]**2
V = 0.

L = simplify(T - V)

In [4]:
dL_dth_s = diff(L, y[0])
dL_dth_1 = diff(L, y[1])
dL_dth_2 = diff(L, y[2])

dL_ddth_s = diff(L, y[3])
dL_ddth_1 = diff(L, y[4])
dL_ddth_2 = diff(L, y[5])

dL_ddth_s_dt = diff(dL_ddth_s, t)
dL_ddth_1_dt = diff(dL_ddth_1, t)
dL_ddth_2_dt = diff(dL_ddth_2, t)

th_s_eqn = Eq(simplify(dL_ddth_s_dt - dL_dth_s), tau[0] - b[0]*y[3])
th_1_eqn = Eq(simplify(dL_ddth_1_dt - dL_dth_1), tau[1] - b[1]*y[4])
th_2_eqn = Eq(simplify(dL_ddth_2_dt - dL_dth_2), tau[2] - b[2]*y[5])

In [5]:
eqs = [th_s_eqn.lhs - th_s_eqn.rhs,
       th_1_eqn.lhs - th_1_eqn.rhs,
       th_2_eqn.lhs - th_2_eqn.rhs]

A, B = linear_eq_to_matrix(eqs, qddot[0], qddot[1], qddot[2])

In [6]:
from sympy import lambdify

f = lambdify((y, tau, rho, m, I , b), A.LUsolve(B), "numpy")

In [9]:
y = np.zeros(6)
u = np.zeros(3)
u[1] = np.deg2rad(30.)

rho = np.array([0.5, 0.5, 0.5])
m = np.array([250., 25., 25.])
I = np.array([25., 2.5, 2.5])
b = np.zeros(3)

print(f(y, u, rho, m, I, b))

[[-0.02223101]
 [ 0.05148234]
 [-0.04680213]]


In [14]:
import dill
# import joblib

with open('lambdified_dynamics.pkl', "wb") as file:
    dill.dump(f, file)

# joblib.dump(f, "lambdified_dynamics.pkl")